# Cleaning of leak data

This notebook contains the code to clean the leak data. 

In [9]:
import pandas as pd
from simpledbf import Dbf5
from janitor import clean_names

In [18]:
# Read raw data
leaks2020_raw = Dbf5("../data/raw/2020_FTC-EJ.dbf").to_dataframe()
leaks2021_raw = Dbf5("../data/raw/2021_FTC-EJ.dbf").to_dataframe()
leaks2022_raw = Dbf5("../data/raw/FTC_2022_EJ.dbf").to_dataframe()

In [8]:
print("Total leaks 2020: ", leaks2020_raw.shape[0])
print("Total leaks 2021: ", leaks2021_raw.shape[0])
print("Total leaks 2022: ", leaks2022_raw.shape[0])
print("Total leaks 20-22:", leaks2020_raw.shape[0] + leaks2021_raw.shape[0] + leaks2022_raw.shape[0])

Total leaks 2020:  595
Total leaks 2021:  373
Total leaks 2022:  267
Total leaks 20-22: 1235


## Merge dfs


In [19]:
leaks_clean = clean_names(pd.concat([leaks2020_raw, leaks2021_raw, leaks2022_raw]))
print("Shape of merged df: ", leaks_clean.shape)
leaks_clean.head()

Shape of merged df:  (1235, 25)


,id,fecha_de_c,fecha_de_i,fecha_fina,codunidad,fechaasign,fechalegal,nombreunid,codestado,nombreesta,...,com2,causal,orden,solicitud,cliente,producto,consecruta,codruta,x,y
0,20445.0,2020-01-16,2020-01-17,2020-01-17,757.0,2020-01-16,2020-01-28,CT - MANTTO. DE REDES,8.0,Cerrada,...,SE REPARO FTC; ARROYO 1.20X.90X.90X.15,"SE REPARA FTC, GENERA BA",13617175.0,NaN,327353.0,327353.0,113.0,SECTOR 11/BLOQUE 02 - RUTA 110106,785784.425000,2.419171e+06
1,20508.0,2019-12-18,2019-12-18,2019-12-30,757.0,2019-12-18,2020-01-08,CT - MANTTO. DE REDES,8.0,Cerrada,...,SE ATENDIO CON FTA,"EJECUTADA, NO GENERA BA",13024028.0,NaN,32281.0,32281.0,85.0,SECTOR 03/BLOQUE 01 - RUTA 302701,779459.868158,2.421359e+06
2,20543.0,2019-12-20,2019-12-23,2019-12-23,757.0,2019-12-20,2020-01-08,CT - MANTTO. DE REDES,8.0,Cerrada,...,SE REPARO FTC; BACHE A CARGO DEL CONTRATISTA.,"EJECUTADA, NO GENERA BA",13100314.0,NaN,341753.0,341753.0,113.0,SECTOR 05/BLOQUE 03 - RUTA 519801,781333.400341,2.424917e+06
3,20574.0,2020-01-06,2020-01-06,2020-01-06,757.0,2020-01-06,2020-01-06,CT - MANTTO. DE REDES,8.0,Cerrada,...,SE REPARO FUGA EN TUBERIA CENTRAL ARROY ...,"SE REPARA FTC, GENERA BA",13360204.0,NaN,72167.0,72167.0,176.0,SECTOR 10/BLOQUE 05 - RUTA MO0035,783703.249350,2.418976e+06
4,20654.0,2020-01-09,2020-01-09,2020-01-21,757.0,2020-01-09,2020-01-21,CT - MANTTO. DE REDES,8.0,Cerrada,...,SE REPARO FUGA EN TOMA,"SE REPARA FTC, GENERA BA",13448539.0,NaN,139505.0,139505.0,22.0,SECTOR 05/BLOQUE 05 - RUTA 517900,781450.023225,2.420611e+06


In [20]:
leaks_clean.columns

Index(['id', 'fecha_de_c', 'fecha_de_i', 'fecha_fina', 'codunidad',
       'fechaasign', 'fechalegal', 'nombreunid', 'codestado', 'nombreesta',
       'codtrabajo', 'nombretrab', 'actividad', 'barrio', 'com1', 'com2',
       'causal', 'orden', 'solicitud', 'cliente', 'producto', 'consecruta',
       'codruta', 'x', 'y'],
      dtype='object')

## Cleaning

In [21]:
# Filter leaks that were repaired
leaks_clean = leaks_clean.query("causal == 'SE REPARA FTC, GENERA BA'")

# Drop irrelevant columns
leaks_clean = leaks_clean.drop(["codunidad", "nombreesta", "nombretrab", "fecha_de_c"], axis = 1)

# Sort df by fecha_de_inicio
leaks_clean = leaks_clean.sort_values(by = "fecha_de_i").reset_index(drop = True)

# Show clean df 
leaks_clean.head() # This df is going to be necessary to define the date of each image to be downloaded

,id,fecha_de_i,fecha_fina,fechaasign,fechalegal,nombreunid,codestado,codtrabajo,actividad,barrio,...,com2,causal,orden,solicitud,cliente,producto,consecruta,codruta,x,y
0,21129.0,2019-12-20,2019-12-20,2020-01-07,2020-01-07,CT - MANTTO. DE REDES,8.0,10208.0,FTC - FUGA EN TUBERIA CENTRAL,OJOCALIENTE FRACC.,...,SE REPARO FUGA EN TUBERIA CENTRAL ...,"SE REPARA FTC, GENERA BA",13371532.0,NaN,116511.0,116511.0,140.0,SECTOR 05/BLOQUE 08 - RUTA 514605,783830.235410,2.422685e+06
1,20960.0,2019-12-30,2019-12-30,2020-01-13,2020-01-13,CT - MANTTO. DE REDES,8.0,10208.0,FTC - FUGA EN TUBERIA CENTRAL,JARD. DE LA CRUZ FRACC.,...,SE REPARARO FUGA EN TUBERUIA CENTRAL ARR...,"SE REPARA FTC, GENERA BA",13506791.0,4655204,86951.0,9704.0,0.0,SECTOR 05/BLOQUE 05 - RUTA 512300,781540.394923,2.421735e+06
2,21270.0,2020-01-03,2020-01-03,2020-01-15,2020-01-15,CT - MANTTO. DE REDES,8.0,10208.0,FTC - FUGA EN TUBERIA CENTRAL,SAN PEDRO CIENEGUILLA,...,SE REPARO FTC; ARROYO 1.40X.80X1.30X.16,"SE REPARA FTC, GENERA BA",13571361.0,NaN,242891.0,242891.0,88.0,SECTOR AR/BLOQUE 03 - RUTA R50500,767561.279127,2.403804e+06
3,20574.0,2020-01-06,2020-01-06,2020-01-06,2020-01-06,CT - MANTTO. DE REDES,8.0,10208.0,FTC - FUGA EN TUBERIA CENTRAL,EMILIANO ZAPATA FRACC.,...,SE REPARO FUGA EN TUBERIA CENTRAL ARROY ...,"SE REPARA FTC, GENERA BA",13360204.0,NaN,72167.0,72167.0,176.0,SECTOR 10/BLOQUE 05 - RUTA MO0035,783703.249350,2.418976e+06
4,20723.0,2020-01-07,2020-01-07,2020-01-17,2020-01-17,CT - MANTTO. DE REDES,8.0,10208.0,FTC - FUGA EN TUBERIA CENTRAL,OJOCALIENTE FRACC.,...,SE REPARO FTC; ARROYO 2.00X1.10X1.10X.20,"SE REPARA FTC, GENERA BA",13660419.0,NaN,135158.0,135158.0,228.0,SECTOR 05/BLOQUE 08 - RUTA 514609,783816.829545,2.422573e+06


In [22]:
# Save clean df 
leaks_clean.to_csv("../data/clean/leaks20-22_clean.csv")